In [1]:
cd ..

C:\Python\weldx


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import asdf
import xarray as xr
import sympy
import pint
import numpy as np
import pandas as pd

import weldx

from weldx import Q_

import weldx.measurement as msm
from weldx.asdf.tags.weldx.core.mathematical_expression import MathematicalExpression

from weldx.asdf.extension import WeldxAsdfExtension, WeldxExtension

## Data

In [4]:
def sine(f, amp, t_end, samples=None):
    """Generate a sine with frequency f, amplitude amp and timespan t_end"""
    if samples is None: # default to 50 samples per period
        samples = int((t_end*2*Q_("Hz")*50).to_reduced_units())+1
    om = 2*np.pi*f
    t = np.linspace(0*Q_("s"),t_end,num=samples)
    y = amp * np.sin(t*om)
    return y,t

In [5]:
I,time = sine(f=1.5*Q_("Hz"), amp=50*Q_("A"), t_end=10*Q_("s"))
time = pd.TimedeltaIndex(data=time.magnitude,unit="s")+pd.Timestamp("2020-06-01")

current_data = msm.Data(
    name="Welding current", data=xr.DataArray(I, dims=["time"],coords={"time":time})
)

In [6]:
V,time = sine(f=1.5*Q_("Hz"), amp=20*Q_("V"), t_end=10*Q_("s"))
time = pd.TimedeltaIndex(data=time.magnitude,unit="s")+pd.Timestamp("2020-06-01")

voltage_data = msm.Data(
    name="Welding voltage", data=xr.DataArray(I, dims=["time"],coords={"time":time})
)

## Software

In [7]:
from asdf.tags.core import Software

In [8]:
twincat_scope = Software(name = "Beckhoff TwinCAT ScopeView", version = "3.4.3143")

## Equipment

In [9]:
HKS_sensor = msm.GenericEquipment(
    "HKS P1000-S3"
)

BH_ELM = msm.GenericEquipment(
    "Beckhoff ELM3002-0000"
)

## Current measurement

In [10]:
src_01 = msm.Source(
    name="Current Sensor",
    output_signal=msm.Signal("analog", "V", data=None),
    error=msm.Error(1337.42),
)

HKS_sensor.sources.append(src_01)

# define AD conversion expression and transformation step
[a, x, b] = sympy.symbols("a x b")
current_ad_func = MathematicalExpression(a * x + b)
current_ad_func.set_parameter("a", Q_(32768.0/10.0,"1/V"))
current_ad_func.set_parameter("b", Q_(0.0,""))

current_AD_transform = msm.DataTransformation(
    name="AD conversion current measurement",
    input_signal=msm.Signal("analog", "V", data=None),
    output_signal=msm.Signal("digital", "", data=None),
    error=msm.Error(0.01),
    func=current_ad_func,
)

HKS_sensor.data_transformations.append(current_AD_transform)

# define current output calibration expression and transformation
current_calib_func = MathematicalExpression(a * x + b)
current_calib_func.set_parameter("a", Q_(1000.0/32768.0,"A"))
current_calib_func.set_parameter("b", Q_(0.0,"A"))

current_calib = msm.DataTransformation(
    name="Calibration current measurement",
    input_signal=msm.Signal("digital", "", data=None),
    output_signal=msm.Signal("digital", "A", data=current_data),
    error=msm.Error(0.0),
    func=current_calib_func,
    meta=twincat_scope
)

HKS_sensor.data_transformations.append(current_calib)

welding_current_chain = msm.MeasurementChain(
    name="welding current measurement", data_source=src_01, data_processors=[current_AD_transform, current_calib]
)

welding_current = msm.Measurement(
    name="welding current measurement", data=[current_data], measurement_chain=welding_current_chain
)

In [11]:
src_02 = msm.Source(
    name="Voltage Sensor",
    output_signal=msm.Signal("analog", "V", data=None),
    error=msm.Error(1),
)

HKS_sensor.sources.append(src_02)

# define AD conversion expression and transformation step
[a, x, b] = sympy.symbols("a x b")
voltage_ad_func = MathematicalExpression(a * x + b)
voltage_ad_func.set_parameter("a", Q_(32768.0/10.0,"1/V"))
voltage_ad_func.set_parameter("b", Q_(0.0,""))

voltage_AD_transform = msm.DataTransformation(
    name="AD conversion voltage measurement",
    input_signal=msm.Signal("analog", "V", data=None),
    output_signal=msm.Signal("digital", "", data=None),
    error=msm.Error(0.01),
    func=voltage_ad_func,
)

HKS_sensor.data_transformations.append(voltage_AD_transform)

# define current output calibration expression and transformation
voltage_calib_func = MathematicalExpression(a * x + b)
voltage_calib_func.set_parameter("a", Q_(100.0/32768.0,"V"))
voltage_calib_func.set_parameter("b", Q_(0.0,"V"))

voltage_calib = msm.DataTransformation(
    name="Calibration voltage measurement",
    input_signal=msm.Signal("digital", "", data=None),
    output_signal=msm.Signal("digital", "V", data=voltage_data),
    error=msm.Error(0.0),
    func=voltage_calib_func,
    meta=twincat_scope
)

HKS_sensor.data_transformations.append(current_calib)


welding_voltage_chain = msm.MeasurementChain(
    name="welding voltage measurement", data_source=src_02, data_processors=[voltage_AD_transform, voltage_calib]
)

welding_voltage = msm.Measurement(
    name="welding voltage measurement", data=[voltage_data], measurement_chain=welding_current_chain
)

In [12]:
equipment = [HKS_sensor, BH_ELM]
measurements = [welding_current, welding_voltage]

tree = {
    "equipment": equipment,
    #"data": measurement_data,
    "measurements": measurements,
    #"expression": expr_01,
    # "measurement_chains": measurement_chains,
    # "data_sources": sources,
    # "data_processors": processors,
}
with asdf.AsdfFile(tree, extensions=[WeldxExtension(), WeldxAsdfExtension()]) as f:
    f.write_to("notebooks/measurement_example.asdf")

In [13]:
f =  asdf.open("notebooks/measurement_example.yaml", extensions=[WeldxExtension(), WeldxAsdfExtension()])
f.tree["measurements"][1].measurement_chain.data_processors[0].func.evaluate(x=Q_(3,"V"))

9830.400000000001 <Unit('dimensionless')>

In [16]:
f.tree["measurements"][0].measurement_chain.data_processors[0].func.evaluate(x=Q_(3,"V"))

9830.400000000001 <Unit('dimensionless')>